In [1]:
from sage.combinat.posets.posets import FinitePoset 
import random

In [6]:
def atkinson(fp):
	binom_coeff = {} #[a,b] ----> a choose b
	n = len(fp._elements)
	
	#Generate Pascal Triangle up to row n

	binom_coeff[str([0,0])] = 1
	binom_coeff[str([0,1])] = 0
	binom_coeff[str([1,0])] = 1
	for i in range(1, n+1):
		for j in range(0, i+1):
			if j == 0:
				binom_coeff[str([i,j])] = 1
			elif j > i:
				binom_coeff[str([i,j])] = 0
			elif i == j:
				binom_coeff[str([i,j])] = 1
			elif j == 1:
				binom_coeff[str([i,j])] = i
			else:
				binom_coeff[str([i,j])] = binom_coeff[str([i-1, j-1])] + binom_coeff[str([i-1, j])]		
	
	# Recursively decompose tree and calculate alpha spectrums

	def decompose(pos, alpha):
		if pos.cardinality() == 1:
			return [1]

		num_elements = pos.cardinality()

		alpha_lower = True

		num_edges = len(pos.cover_relations())
		
		covers = pos.upper_covers(alpha)

		if len(covers) == 0:
			alpha_lower = False
			covers = pos.lower_covers(alpha)

		beta = random.choice(covers)

		edges = pos.cover_relations()

		if alpha_lower:
			edges.remove([alpha, beta])

		else:
			edges.remove([beta,alpha])
		# Separate posets

		new_pos = FinitePoset(pos._elements, edges)

		sub_pos = new_pos.connected_components()

		# Recurse and recombine

		# Determine which connected component contains alpha 
		if alpha in sub_pos[0]._elements:
			P = sub_pos[0]
			Q = sub_pos[1]

		else:
			P = sub_pos[1]
			Q = sub_pos[0]

		u = P.cardinality()
		v = Q.cardinality()

		P_spec = decompose(P, alpha)
		Q_spec = decompose(Q, beta)

		# Calculate alpha spec 

		alpha_spec = []
		sums_Q_spec = [Q_spec[0]]

		for i in range(1, len(Q_spec)):
			sums_Q_spec.append(sums_Q_spec[i-1] + Q_spec[i])

		# If alpha is not maximal, calculate formula 1
		if alpha_lower:
			for rank in range(0, num_elements):
				r = rank + 1
				rank_r_val = 0
				for ind in range(max(1, r - v)-1, min(u, r)):
					i = ind + 1
					k = binom_coeff[str([r-1, i-1])] * binom_coeff[str([u+v-r, u-i])]
					rank_r_val += P_spec[ind] * k * (sums_Q_spec[v-1] - sums_Q_spec[r - i])
				alpha_spec.append(rank_r_val)
		# Otherwise, calculate formula 2, where beta < alpha
		else:
			for rank in range(0, num_elements):
				r = rank + 1
				rank_r_val = 0
				for ind in range(max(1, r - v)-1, min(u, r)):
					i = ind + 1
					k = binom_coeff[str([r-1, i-1])] * binom_coeff[str([u+v-r, u-i])]
					rank_r_val += P_spec[ind] * k * (sums_Q_spec[r-i - 1])
				alpha_spec.append(rank_r_val)





	l = decompose(fp, random.choice(fp._elements))

	return sum(l)


In [7]:
D = DiGraph([[0,1,2,3,4,5,6,7,8,9],[[0,1],[2,1],[2,3],[4,3],[4,5],[6,5],[7,4],[7,8],[9,8]]])
atkinson(FinitePoset(D))

AssertionError: 